In [53]:
import numpy as np
import madmom
import pickle
from madmom.ml.nn import NeuralNetwork
from copy import deepcopy
from modules.madmom_cnn_prep import cnn_preprocessor
import tensorflow as tf

tf.keras.backend.set_floatx("float32")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
def cnn_normalize(frames):
    inv_std = np.load("models/bock2013pret_inv_std.npy")
    mean = np.load("models/bock2013pret_mean.npy")
    frames_normalized = (frames - np.reshape(mean, (1,80,3)))*np.reshape(inv_std, (1,80,3))
    return frames_normalized

In [55]:
# Data
# Preprocess and normalize - copied from MadMom
preprocessor = cnn_preprocessor()
frames = preprocessor("datasets/OnsetLabeledInstr2013/development/Violin/42954_FreqMan_hoochie_violin_pt1.wav")
frames_normalized = cnn_normalize(frames)

In [56]:
# Madmom model object
with open('datasets/madmom_models-master/onsets/2013/onsets_cnn.pkl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    mm_model = u.load()
mm_model.layers


In [57]:
my_model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=frames.shape),
        tf.keras.layers.Conv2D(
            activation = 'tanh',
            filters = 10,
            kernel_size = (7,3),
            strides = 1,
            trainable = False
        ),
    ])

# Crucial step --- load weights from MadMom object

my_model.layers[0].set_weights([
    np.transpose(mm_model.layers[1].weights, [2,3,0,1]), 
    mm_model.layers[1].bias
])

In [58]:
with open('datasets/madmom_models-master/onsets/2013/onsets_cnn.pkl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    mm_model = u.load()

# Process for altering MadMom model - create new pickle file
# Keep only normalization layer

mm_model.layers =[mm_model.layers[0]]
with open('models/temp_model.pickle', 'wb') as file_pi:
    pickle.dump(mm_model, file_pi)
altered_nn = NeuralNetwork.load('models/temp_model.pickle')



In [59]:
# Compare
arr1 = altered_nn(frames)
arr2 = frames_normalized
for arr in [arr1, arr2]:
    print(arr.shape, arr.sum())


(1070, 80, 3) 60564.552902900556
(1070, 80, 3) 60564.552902900556


In [60]:
with open('datasets/madmom_models-master/onsets/2013/onsets_cnn.pkl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    mm_model = u.load()

# Process for altering MadMom model - create new pickle file
# Keep only normalization layer 
# + first convolution layer

mm_model.layers = mm_model.layers[0:2]
with open('models/temp_model.pickle', 'wb') as file_pi:
    pickle.dump(mm_model, file_pi)
altered_nn = NeuralNetwork.load('models/temp_model.pickle')

In [61]:
# Compare
arr1 = my_model.predict(frames_normalized.reshape((1,-1,80,3))).reshape(-1,78,10)
arr2 = altered_nn(frames)
for arr in [arr1, arr2]:
    print(arr.shape, arr.sum())

print("Sum of absolute difference: ", np.sum(np.abs(arr1-arr2)))

(1064, 78, 10) -445896.66
(1064, 78, 10) -450595.38
Sum of absolute difference:  343635.25


In [66]:
# Why are they different? Explore the biases and weights:

w1 = mm_model.layers[1].bias
w2 = my_model.layers[0].get_weights()[1]

print("Sum of bias tensors before and after TF conversion:")
print(np.sum(w1))
print(np.sum(w2))
print("Sum of absolute difference: ", np.sum(np.abs(w1-w2)))
print("All close? ", np.allclose(w1, w2, rtol=0, atol=1e-20))

Sum of bias tensors before and after TF conversion:
-11.930101
-11.930101
Sum of absolute difference:  0.0
All close?  True


In [65]:
# Weigths:

w1 = np.transpose(mm_model.layers[1].weights, [2,3,0,1])
w2 = my_model.layers[0].get_weights()[0]

print("Sum of weight tensors before and after TF conversion:")
print(np.sum(w1))
print(np.sum(w2))
print("Sum of absolute difference: ", np.sum(np.abs(w1-w2)))
print("All close? ", np.allclose(w1, w2, rtol=0, atol=1e-20))

Sum of weight tensors before and after TF conversion:
0.90992165
0.90991974
Sum of absolute difference:  0.0
All close?  True
